In [1]:
using JuMP
using Ipopt
using Complementarity

In [5]:
G = Dict()

pi = .01
L = .5
σ = .9
kvalue = 0
nfirm = 1

ρ = 1 - 1/σ
gammamax = pi*(1-L)^(-1/σ)/(pi*(1-L)^(-1/σ) +1 - pi)

0.021353469024510078

In [28]:
mcp_model = MCPModel()

@variables(mcp_model,begin
    P_G>=1e-5, (start=1-pi,)	#Price index for consumption on a good day,
    P_B>=1e-5, (start=pi,)	#Price index for consumption on a bad day,
    C_G, (start=1-pi + pi*(1-L),)	#Consumption in the good day,
    C_B, (start=1-pi + pi*(1-L),)	#Consumption in the bad day,
    P_C>=1e-5, (start=1,)	#Consumption price index,
    M, (start=1-pi + pi*(1-L),)	#Income,
    K>=0, (start = (1-pi + pi*(1-L) - (1-L))/(1-pi),)	#Coverage
    GAMMA>=0	#Premium for coverage
    PROFIT	#Firm profit
end)


@NLexpressions(mcp_model,begin
    income, M - P_G - P_B*(1-L)
    P_Cdef, P_C - ( (1-pi)*(P_G/(1-pi))^(1-σ) + pi * (P_B/pi)^(1-σ))^(1/(1-σ))
    C_Gdef, C_G - ( M/P_C * (P_C*(1-pi)/P_G)^σ)
    C_Bdef, C_B - ( M/P_C * (P_C*pi/P_B)^σ)
    market_G, 1 - ( C_G + GAMMA*K)
    market_B, 1 - L + K - ( C_B + GAMMA*K)
    demand_K, GAMMA*(P_G+P_B) - P_B
    profitdef, PROFIT - (GAMMA - pi) * (K-kvalue*(nfirm-1)/nfirm)
    gamma_eq, 0
end)

@complementarity(mcp_model,income,M)
@complementarity(mcp_model,P_Cdef,P_C)
@complementarity(mcp_model,market_G,P_G)
@complementarity(mcp_model,market_B,P_B)
@complementarity(mcp_model,C_Gdef,C_G)
@complementarity(mcp_model,C_Bdef,C_B)
@complementarity(mcp_model,demand_K,K)
@complementarity(mcp_model,profitdef,PROFIT)
@complementarity(mcp_model,gamma_eq,GAMMA)


fix(GAMMA,pi,force=true)

solveMCP(mcp_model)

mcp_sol = Dict(
    :P_G=> .99,
    :P_B=> .01,
    :C_G=> .995,
    :C_B=> .995,
    :P_C=> 1,
    :M=> .995,
    :K=> .5,
    :GAMMA=> .01,
    :PROFIT=> 0
)

for (var,sol) in mcp_sol
    @assert isapprox(sol,result_value(mcp_model[var]),atol=1e-8) "Variable, $var, doesn't match in 13-NashInsurance"
end



Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.0000e+20             0.0e+00 (profitdef)
    1     1     1     8 9.0909e+19  1.0e+00    1.0e+01 (profitdef)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 9.0909e+19           I 9.0e+00 9.1e+19 (profitdef)
    1     1     3     3 7.1146e+19  1.0e+00 SO 3.6e+00 7.1e+19 (profitdef)
    2     1     4     4 4.1988e+19  1.0e+00 SO 1.4e+00 4.2e+19 (profitdef)
    3     1     5     5 1.5346e+19  1.0e+00 SO 5.8e-01 1.5e+19 (profitdef)
    4     1     6     6 2.8736e+18  1.0e+00 SO 2.3e-01 2.9e+18 (profitdef)
    5     1     7     7 2.4248e+17  1.0e+00 SO 9.2e-02 2.4e+17 (profitdef)
    6     1     8     8 8.6211e+15  1.0e+00 SO 3.7e-02 8.6e+15 (profitdef)
 

In [46]:
function Nash()

    model = Model(Ipopt.Optimizer)

    @variables(model,begin
        P_G>=1e-5, (start=1-pi,)	#Price index for consumption on a good day,
        P_B>=1e-5, (start=pi,)	#Price index for consumption on a bad day,
        C_G, (start=1-pi + pi*(1-L),)	#Consumption in the good day,
        C_B, (start=1-pi + pi*(1-L),)	#Consumption in the bad day,
        P_C>=1e-5, (start=1,)	#Consumption price index,
        M, (start=1-pi + pi*(1-L),)	#Income,
        K>=0, (start = (1-pi + pi*(1-L) - (1-L))/(1-pi),)	#Coverage
        0<=GAMMA<=gammamax, (start=gammamax/2,)	#Premium for coverage
        PROFIT	#Firm profit
    end)


    @NLconstraints(model,begin
        income, M == P_G + P_B*(1-L)
        P_Cdef, P_C == ( (1-pi)*(P_G/(1-pi))^(1-σ) + pi * (P_B/pi)^(1-σ))^(1/(1-σ))
        C_Gdef, C_G == ( M/P_C * (P_C*(1-pi)/P_G)^σ)
        C_Bdef, C_B == ( M/P_C * (P_C*pi/P_B)^σ)
        market_G, 1 == ( C_G + GAMMA*K)
        market_B, 1 - L + K == ( C_B + GAMMA*K)
        demand_K, GAMMA*(P_G+P_B) == P_B
        profitdef, PROFIT == (GAMMA - pi) * (K-kvalue*(nfirm-1)/nfirm)
    end)

    @objective(model,Max,PROFIT)

    return model

end

Nash (generic function with 1 method)

In [66]:
kvalue = result_value(mcp_model[:K])
iterlog = Dict()
equil = Dict()

for n = 1:15

    dev = 1
    nfirm = n

    for iter = 1:25

        N = Nash()
        set_silent(N)
        optimize!(N)

        iterlog[n,iter] = Dict()

        iterlog[n,iter][:dev] = dev;
        iterlog[n,iter][:K] =value(N[:K]);
        iterlog[n,iter]["K/L"] =value(N[:K])/L;
        iterlog[n,iter][:GAMMA] =value(N[:GAMMA]);
        iterlog[n,iter][:PROFIT] = (value(N[:GAMMA]) - pi) *value(N[:K]);
        iterlog[n,iter]["P_B/pi"] =value(N[:P_B])/pi;

        equil[n] = Dict()
        equil[n][:dev] = dev;
        equil[n][:K] =value(N[:K]);
        equil[n]["K/L"] =value(N[:K])/L;
        equil[n][:GAMMA] =value(N[:GAMMA]);
        equil[n][:PROFIT] = (value(N[:GAMMA]) - pi) *value(N[:K]);
        equil[n]["P_B/pi"] =value(N[:P_B])/pi;
        equil[n]["C_G"] = value(N[:C_G])
        equil[n]["C_B"] = value(N[:C_B])

        dev = abs(kvalue - value(N[:K]));
        kvalue = value(N[:K])

        if dev < 1e-5
            break
        end

    end

end


In [67]:
iterlog[1,1]

Dict{Any, Any} with 6 entries:
  "P_B/pi" => 7.49876
  :PROFIT  => 0.000634239
  :K       => 0.364902
  :GAMMA   => 0.0117381
  :dev     => 1
  "K/L"    => 0.729803

In [68]:
equil[1]

Dict{Any, Any} with 8 entries:
  "P_B/pi" => 7.49876
  "C_B"    => 0.860618
  :PROFIT  => 0.000634239
  :K       => 0.364902
  :GAMMA   => 0.0117381
  "C_G"    => 0.995717
  :dev     => 0.135098
  "K/L"    => 0.729803

In [65]:
sort(collect(keys(iterlog)))

352-element Vector{Any}:
 (1, 1)
 (1, 2)
 (2, 1)
 (2, 2)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (2, 7)
 (2, 8)
 ⋮
 (15, 17)
 (15, 18)
 (15, 19)
 (15, 20)
 (15, 21)
 (15, 22)
 (15, 23)
 (15, 24)
 (15, 25)